# Cume Dist and Percent Rank

Currently Percent rank implemented by SQL Databases and Pandas Percent rank use different algorithms.

Pandas Percent Rank uses the same algoritm used by SQL Databases CumeDist function.

The calculation used by SQL Databases for CumeDist and Percent rank are:


$$
PERCENT\_RANK = (RANK – 1)/(COUNT -1)
$$

$$
CUME\_DIST = RANK/COUNT
$$


Current, [Ibis](http://ibis-project.org) uses the approach of Pandas, but probably should use the same 
behavior used by the SQL Databases.

More information about the difference between Percent Rank and CumeDist [here](https://www.sqlservercentral.com/articles/whats-the-difference-between-percent_rank-and-cume_dist)


Some Ibis reference about the current implementation:

- https://github.com/ibis-project/ibis/blob/1.3.0/ibis/pandas/execution/window.py#L360
- https://github.com/ibis-project/ibis/blob/1.3.0/ibis/tests/all/test_window.py#L41


This notebook aims to implement a function for sql percent_rank using pandas and test it against 
[OmniSciDB](https://docs.omnisci.com/).

In [1]:
from copy import copy

import ibis
import pandas as pd
# local
from settings import conf
from utils import cursor2df

### Setup

In [2]:
BACKENDS = ['omniscidb']
con = {
    backend: getattr(ibis, backend).connect(**conf[backend]) 
    for backend in BACKENDS
}

In [10]:
t = con['omniscidb'].table('functional_alltypes')

ref_0
DatabaseTable[table]
  name: functional_alltypes
  schema:
    index : int64
    Unnamed__0 : int64
    id : int32
    bool_col : boolean
    tinyint_col : int16
    smallint_col : int16
    int_col : int32
    bigint_col : int64
    float_col : float32
    double_col : float64
    date_string_col : string
    string_col : string
    timestamp_col : timestamp
    year_ : int32
    month_ : int32

index = Column[int64*] 'index' from table
  ref_0


### Percent Rank and CumeDist definition

In [ ]:
def win_count(se):
    count = copy(se)
    count[:] = se.count()
    return count


def sql_percent_rank(se):
    """
    PERCENT_RANK = (RANK – 1)/(COUNT -1)
    """
    return (se.rank(method='min') - 1) / (se.apply(win_count) - 1)


def sql_cume_dist(se):
    """
    CUME_DIST = RANK/COUNT
    """
    return se.rank(method='min') / se.apply(win_count)


def pd_percent_rank(se): 
    return se.rank(method='min', pct=True)

### Test CUME DIST using Ibis/OmniSciDB

In [ ]:
alltypes = t.execute()

analytic_alltypes = alltypes.sort_values('id').groupby('string_col')

result = alltypes.assign(
    pandas_pct_rank=pd_percent_rank(analytic_alltypes.id),
    cume_dist=sql_cume_dist(analytic_alltypes.id),
    pct_rank=sql_percent_rank(analytic_alltypes.id),
).set_index('id').sort_index()

# display(result[['pandas_pct_rank', 'cume_dist']].describe())
pd.testing.assert_series_equal(
    result['pandas_pct_rank'], result['cume_dist'],
    check_dtype=False,
    check_names=False,
)

### Test OmniSciDB Percent Rank

In [ ]:
sql = '''
SELECT 
    id, cume_dist() OVER (PARTITION BY "string_col" ORDER BY "id") AS cume_dist_val,
    percent_rank() OVER (PARTITION BY "string_col" ORDER BY "id") AS percent_rank_val
FROM functional_alltypes
'''

cur = con['omniscidb'].con.execute(sql)
df = cursor2df(cur).set_index('id').sort_index()
df.head()

In [ ]:
pd.testing.assert_series_equal(
    result['cume_dist'], 
    df['cume_dist_val'],
    check_dtype=False,
    check_names=False,
)

pd.testing.assert_series_equal(
    result['cume_dist'], 
    df['cume_dist_val'],
    check_dtype=False,
    check_names=False,
)

### MySQL

In [ ]:
con['mysql'] = ibis.mysql.connect(**conf['mysql'])

sql = '''
SELECT 
    t0.`index`, t0.`Unnamed: 0`, t0.id, t0.bool_col, 
    t0.tinyint_col, t0.smallint_col, t0.int_col, 
    t0.bigint_col, t0.float_col, t0.double_col, 
    t0.date_string_col, t0.string_col, 
    t0.timestamp_col, t0.year, t0.month, 
    percent_rank() OVER (PARTITION BY t0.string_col ORDER BY t0.id ASC ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) AS val 
FROM (
    SELECT t1.`index` AS `index`, t1.`Unnamed: 0` AS `Unnamed: 0`, t1.id AS id, 
    t1.bool_col = 1 AS bool_col, t1.tinyint_col AS tinyint_col, t1.smallint_col AS smallint_col, 
    t1.int_col AS int_col, t1.bigint_col AS bigint_col, t1.float_col AS float_col, 
    t1.double_col AS double_col, t1.date_string_col AS date_string_col, t1.string_col AS string_col, 
    t1.timestamp_col AS timestamp_col, t1.year AS year, t1.month AS month 
FROM functional_alltypes AS t1) AS t0 
LIMIT 10000
'''

con['mysql'].con.execute(sql)

In [ ]:
sql = '''
SELECT 
    t0.`index`, t0.`Unnamed: 0`, t0.id, t0.bool_col, t0.tinyint_col,
    t0.smallint_col, t0.int_col, t0.bigint_col, t0.float_col, t0.double_col,
    t0.date_string_col, t0.string_col, t0.timestamp_col, t0.year, t0.month,
    percent_rank() OVER (PARTITION BY t0.string_col ORDER BY t0.id ASC) AS val
FROM (
    SELECT t1.`index` AS `index`, t1.`Unnamed: 0` AS `Unnamed: 0`, t1.id AS id, 
        t1.bool_col = 1 AS bool_col, 
        t1.tinyint_col AS tinyint_col, t1.smallint_col AS smallint_col, 
        t1.int_col AS int_col, t1.bigint_col AS bigint_col, t1.float_col AS float_col,
        t1.double_col AS double_col, t1.date_string_col AS date_string_col,
        t1.string_col AS string_col, t1.timestamp_col AS timestamp_col,
        t1.year AS year, t1.month AS month
    FROM functional_alltypes AS t1
) AS t0
'''
con['mysql'].con.execute(sql)

## Conclusions

This notebook implemented Percent Rank and CumeDist for pandas. The tests against OmniSciDB passed with success.  